In [ ]:
# 🚀 Обзор FastAPI проекта

Добро пожаловать в интерактивное введение в наш FastAPI проект! Этот notebook познакомит вас с основными компонентами системы.

## 📚 Что вы изучите
- Архитектуру проекта и его компоненты
- Основные API endpoints
- Системы аутентификации и realtime
- Настройку окружения

## 🎯 Предварительные требования
- Python 3.11+
- Базовые знания FastAPI
- Запущенная инфраструктура (PostgreSQL, RabbitMQ, Redis)


In [ ]:
## 🏗️ Архитектура проекта

Наш проект построен по модульной архитектуре:


In [ ]:
import os
import sys
from pathlib import Path

# Добавляем корневую папку проекта в путь
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"📁 Корневая папка проекта: {project_root}")
print(f"🐍 Python версия: {sys.version}")
print(f"📍 Текущая директория: {os.getcwd()}")


In [ ]:
# Визуализация структуры проекта
import ipywidgets as widgets
from IPython.display import display, HTML

def show_project_structure():
    structure = """
    <div style='font-family: monospace; background: #f8f9fa; padding: 15px; border-radius: 5px;'>
    <h3>📂 Структура проекта</h3>
    <pre>
src/
├── 📱 apps/                   # Бизнес-логика приложений
│   ├── 🔐 auth/              # Система аутентификации
│   └── 👤 users/             # Управление пользователями
├── ⚙️ core/                   # Основная инфраструктура
│   ├── 📊 base/              # Базовые компоненты
│   ├── 🗄️ database.py        # Подключение к БД
│   ├── 📨 messaging/         # RabbitMQ система
│   ├── 🔄 realtime/          # WebSocket + SSE
│   ├── 🌊 streaming/         # Streaming клиенты
│   ├── ⚡ taskiq_client.py   # Фоновые задачи
│   └── 🤖 telegram/          # Telegram боты
├── 🛠️ tools/                  # Инструменты и утилиты
│   ├── 📡 clients/           # WS/SSE клиенты
│   └── 🔄 migrations/        # Система миграций
└── 🚀 main.py               # Точка входа FastAPI
    </pre>
    </div>
    """
    return HTML(structure)

show_project_structure()


In [ ]:
## 🔧 Настройка окружения

Давайте проверим, что все необходимые компоненты настроены:


In [ ]:
# Проверка переменных окружения
import os
from dotenv import load_dotenv

# Загружаем переменные окружения
load_dotenv(project_root / '.env')

# Список критически важных переменных
critical_vars = [
    'DATABASE_URL',
    'SECRET_KEY',
    'JWT_SECRET_KEY',
    'RABBITMQ_URL',
    'REDIS_URL'
]

print("🔍 Проверка переменных окружения:")
print("=" * 40)

for var in critical_vars:
    value = os.getenv(var)
    status = "✅" if value else "❌"
    masked_value = f"{value[:20]}..." if value and len(value) > 20 else value
    print(f"{status} {var}: {masked_value or 'НЕ НАЙДЕНА'}")

# Интерактивная настройка
if not all(os.getenv(var) for var in critical_vars):
    print("\n⚠️ Некоторые переменные не настроены!")
    print("📝 Создайте файл .env в корне проекта или запустите: make setup-env")


In [ ]:
## 🌐 Основные API Endpoints

Давайте изучим доступные API endpoints интерактивно:


In [ ]:
# Интерактивный explorer API endpoints
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

# Данные о endpoints
api_endpoints = {
    "🔐 Аутентификация": {
        "POST /auth/register": "Регистрация нового пользователя",
        "POST /auth/login": "Вход в систему с получением JWT токенов",
        "POST /auth/refresh": "Обновление access токена",
        "POST /auth/logout": "Выход из системы"
    },
    "👤 Пользователи": {
        "GET /users/profile": "Получение профиля текущего пользователя",
        "PUT /users/profile": "Обновление профиля",
        "GET /users/{user_id}": "Получение пользователя по ID",
        "GET /users/": "Список всех пользователей (админ)"
    },
    "🔄 Realtime": {
        "WS /realtime/ws": "Основной WebSocket endpoint",
        "GET /realtime/events": "Server-Sent Events поток",
        "WS /realtime/webrtc/signaling": "WebRTC сигналинг",
        "POST /realtime/send": "Отправка сообщения через API"
    },
    "📨 Messaging": {
        "POST /messaging/user-notification": "Отправка уведомления пользователю",
        "POST /messaging/admin-notification": "Админское уведомление",
        "GET /messaging/health": "Проверка состояния RabbitMQ"
    }
}

# Создаем интерактивные виджеты
category_dropdown = widgets.Dropdown(
    options=list(api_endpoints.keys()),
    description='Категория:',
    style={'description_width': 'initial'}
)

endpoint_dropdown = widgets.Dropdown(
    options=[],
    description='Endpoint:',
    style={'description_width': 'initial'}
)

description_area = widgets.HTML(value='Выберите категорию и endpoint')

def update_endpoints(change):
    category = change['new']
    endpoints = list(api_endpoints[category].keys())
    endpoint_dropdown.options = endpoints
    if endpoints:
        endpoint_dropdown.value = endpoints[0]

def update_description(change):
    if endpoint_dropdown.value:
        category = category_dropdown.value
        endpoint = endpoint_dropdown.value
        desc = api_endpoints[category][endpoint]
        
        html_content = f"""
        <div style='background: #e8f4fd; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h4 style='color: #0066cc; margin: 0 0 10px 0;'>{endpoint}</h4>
            <p style='margin: 0; color: #333;'>{desc}</p>
        </div>
        """
        description_area.value = html_content

# Подключаем обработчики
category_dropdown.observe(update_endpoints, names='value')
endpoint_dropdown.observe(update_description, names='value')

# Инициализируем первые значения
update_endpoints({'new': category_dropdown.value})

print("🌐 Интерактивный обзор API endpoints:")
display(category_dropdown, endpoint_dropdown, description_area)


In [ ]:
## ✅ Чек-лист изученного

Отметьте, что вы изучили в этом notebook:


In [ ]:
# Интерактивный чек-лист
import ipywidgets as widgets
from IPython.display import display

checklist_items = [
    "📂 Понимаю структуру проекта",
    "⚙️ Настроил переменные окружения", 
    "🌐 Знаю основные API endpoints",
    "🗄️ Проверил подключение к базе данных",
    "🧪 Протестировал компоненты системы",
    "🎯 Выбрал следующий туториал для изучения"
]

checkboxes = []
for item in checklist_items:
    checkbox = widgets.Checkbox(
        value=False,
        description=item,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    checkboxes.append(checkbox)

progress_label = widgets.HTML(value="<h4>📊 Прогресс: 0/6 завершено</h4>")

def update_progress(change):
    completed = sum(1 for cb in checkboxes if cb.value)
    total = len(checkboxes)
    percentage = int((completed / total) * 100)
    
    color = "#4CAF50" if completed == total else "#2196F3"
    progress_label.value = f"<h4 style='color: {color};'>📊 Прогресс: {completed}/{total} завершено ({percentage}%)</h4>"
    
    if completed == total:
        progress_label.value += "<p style='color: #4CAF50; font-weight: bold;'>🎉 Поздравляем! Вы завершили обзор проекта!</p>"

# Подключаем обработчики
for checkbox in checkboxes:
    checkbox.observe(update_progress, names='value')

print("✅ Отметьте изученные разделы:")
display(progress_label)
for checkbox in checkboxes:
    display(checkbox)


In [ ]:
## 🎯 Следующие шаги

Теперь, когда вы знакомы с архитектурой проекта, выберите один из следующих туториалов:

### 🔐 Аутентификация
**Файл:** `../auth/01_user_registration.ipynb`  
**Описание:** Изучите систему аутентификации: регистрация, вход, JWT токены  
**Уровень:** 🟡 Средний | **Время:** 15-20 минут

### 💾 База данных  
**Файл:** `../database/01_models_and_schemas.ipynb`  
**Описание:** Работа с моделями SQLAlchemy и Pydantic схемами  
**Уровень:** 🟢 Начинающий | **Время:** 10-15 минут

### 🔄 Realtime
**Файл:** `../realtime/01_websocket_basics.ipynb`  
**Описание:** WebSocket соединения и обмен сообщениями в реальном времени  
**Уровень:** 🔴 Продвинутый | **Время:** 20-25 минут

### 🧪 Тестирование
**Файл:** `../testing/01_unit_tests.ipynb`  
**Описание:** Написание эффективных тестов с pytest и фабриками данных  
**Уровень:** 🟡 Средний | **Время:** 15-20 минут

## 🔗 Полезные ссылки

- 📚 [Документация проекта](../../docs/README.md)
- 🐳 [Docker настройка](../../README_DOCKER.md) 
- 🧪 [Руководство по тестированию](../../docs/README_tests.md)
- 🔧 [Makefile команды](../../Makefile)
- 🌐 [FastAPI документация](https://fastapi.tiangolo.com/)

---

**Следующий шаг:** Выберите один из туториалов выше и продолжите изучение! 🚀
